In [14]:
import joblib
import pandas as pd
import sqlite3 as sq

import requests
import time

from src.db import (
    execute_fetchall,
    get_schema,
    get_schemas,
    get_schemas,
    drop_table,
    get_tables,
    count_tables,
    get_table,
    update_db
)

from src.format import (
    format_anime_base,
    format_X,
    format_anime_X,
    format_anime_title,
    format_anime,
    format_all_staff
)

In [3]:
db = 'animedb.sqlite'

In [4]:
df_va = get_table('voiceactor', db = db)

In [9]:
subset_columns_update = [
    'website_url',
    'given_name',
    'family_name',
    'birthday',
    'favorites',
    'about'
]

idx_no_update = df_va\
    .dropna(how = 'all', subset = subset_columns_update)\
    ['voiceactor_id']

df_va_request = df_va[~df_va.isin(idx_no_update)]

In [10]:
df_va_request

In [16]:
vas = []
for va in df_va_request['voiceactor_id']:
    try:
        print(len(vas))
        response = requests.get(f'https://api.jikan.moe/v4/people/{va}')
        vas.append(response.json())
        if (len(vas)%500) == 0:
            joblib.dump(vas, f'vas-{len(vas)}.pkl')
        time.sleep(1)
    except:
        print(f'Exception: {va}')
        vas.append([va, None])

In [17]:
joblib.dump(vas, f'vas.pkl')